In [4]:
!pip install qiskit-nature
!pip install --prefer-binary pyscf
!pip install sparse


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.9/306.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/

In [6]:
from qiskit_nature.units import DistanceUnit 
from qiskit_nature.second_q.drivers import PySCFDriver
#used to compute 1body and 2body integrals in 
# electronic-orbital basis, necessary -> Fermionic operator

driver = PySCFDriver(
    atom = 'H .0 .0 .0; H .0 .0 0.735',
    unit = DistanceUnit.ANGSTROM, #Distance Unit to convey dist between atoms/elements
    basis='sto3g',
)

problem = driver.run() # starts driver to simulate

#map to parities
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=problem.num_particles) #for each particle

from qiskit_algorithms.optimizers import L_BFGS_B

optimizer = L_BFGS_B()

from qiskit.primitives import Estimator
estimator = Estimator()

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

#circuits take num spatial orbitals, num particles, and
# require a mapper
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles, 
        mapper,
    )
)

from qiskit_algorithms import VQE
vqe = VQE(estimator, ansatz, optimizer)
#zeros all initial state
vqe.initial_point = [0] * ansatz.num_parameters

from qiskit_nature.second_q.algorithms import GroundStateEigensolver
algorithm = GroundStateEigensolver(mapper, vqe)
electronic_structure_result = algorithm.solve(problem)
electronic_structure_result.formatting_precision = 6
print(electronic_structure_result)


/home/codespace/.python/current/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.857275
  - computed part:      -1.857275
~ Nuclear repulsion energy (Hartree): 0.719969
> Total ground state energy (Hartree): -1.137306
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  1.388949]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  1.388949]
    - computed part:      [0.0  0.0  1.388949]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 
